In [1]:
import os
from datasets import load_from_disk

# 1. 경로 설정
# 원본 경로
original_path = "/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer_bace"
# 새로 저장할 경로 (덮어쓰기 위해 _remake 이름 사용)
save_path_remake = "/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer_bace_remake"

# 2. 템플릿 및 설정
bace_templates = [
    "<INPUT> Based on the molecule given above, suggest the biological activity against BACE-1.",
    "Based on the given molecule: <INPUT>, what biological activity could potentially be observed against BACE-1?",
    "Given the following molecule, please provide the biological activity against BACE-1. <INPUT>",
    "<INPUT> Given the above molecule, what could be the probable biological activity against BACE-1?",
    "Please provide the biological activity value for this molecule against BACE-1: <INPUT>.",
    "Consider that for this molecule, if <INPUT> is given, what is the biological activity against BACE-1?",
    "Propose the biological activity value given this molecule against BACE-1. <INPUT>",
    "Predict the biological activity of the molecule <INPUT> against BACE-1.",
    "Can you tell me the biological activity of the molecule that uses <INPUT> against BACE-1?",
    "Using <INPUT> as the molecule, tell me the biological activity against BACE-1.",
    "Predict the possible biological activity against BACE-1 for the listed molecule. <INPUT>",
    "<INPUT> Considering the given molecule, what might be the biological activity against BACE-1?",
    "A molecule <INPUT> is given; what could be the biological activity against BACE-1?",
]

system_prompt = "You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule captioning, molecule generation."
graph_tokens = "<GRAPH>" + "<mol>" * 32 + "</GRAPH>"

# 3. 로직 함수
def reconstruct_exact_with_space(batch):
    corrected_batch = {key: [] for key in batch.keys()}
    
    for i in range(len(batch['input_mol_string'])):
        mol_str = batch['input_mol_string'][i]
        original_prompt = batch['prompt_text'][i]
        mol_content = mol_str + graph_tokens
        
        found_template_idx = -1
        matched_instruction = ""
        
        # 템플릿 매칭
        for t_idx, template in enumerate(bace_templates):
            candidate_instruction = template.replace("<INPUT>", mol_content)
            if candidate_instruction in original_prompt:
                found_template_idx = t_idx
                matched_instruction = candidate_instruction
                break
        
        # [핵심 수정] 원본 데이터에 맞춰 끝에 공백(" ") 추가
        if found_template_idx != -1:
            # f-string의 맨 끝에 스페이스바를 하나 넣었습니다.
            new_full_prompt = f"<s>[INST] {system_prompt} \n\n{matched_instruction} [/INST] "
        else:
            # 매칭 실패 시 원본 유지 (안전장치)
            new_full_prompt = original_prompt

        corrected_batch['prompt_text'].append(new_full_prompt)
        
        # 나머지 데이터 복사 (prompt_text 제외)
        for key in batch.keys():
            if key == 'prompt_text': continue
            corrected_batch[key].append(batch[key][i])
            
    return corrected_batch

# 4. 실행
print("Loading Original Dataset...")
ds = load_from_disk(original_path)

print("Reconstructing with TRAILING SPACE...")
ds_remake = ds.map(
    reconstruct_exact_with_space,
    batched=True,
    batch_size=1000,
    remove_columns=ds.column_names,
    desc="Processing"
)

ds_remake.save_to_disk(save_path_remake)
print(f"Saved to: {save_path_remake}")

/home/jovyan/miniconda3/envs/MolDA_CHJ/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Original Dataset...
Reconstructing with TRAILING SPACE...


Saving the dataset (1/1 shards): 100%|██████████| 152/152 [00:00<00:00, 16552.45 examples/s]

Saved to: /home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer_bace_remake


In [2]:
from datasets import  load_from_disk

all_templates = load_from_disk('/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer_bace_remake')
len(all_templates)

152

# 원본과 자체제작 본 비교 코드

In [3]:
import os
from datasets import load_from_disk
import difflib

# ==========================================
# 1. 경로 설정
# ==========================================
# 원본 경로
original_path = "/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer_bace"
# 수정된 경로
corrected_path = "/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer_bace_remake"

# ==========================================
# 2. 데이터 로드
# ==========================================
print(f"Loading Original: {original_path}")
ds_orig = load_from_disk(original_path)

print(f"Loading Corrected: {corrected_path}")
ds_new = load_from_disk(corrected_path)

# ==========================================
# 3. 기본 정보 비교
# ==========================================
print("\n" + "="*50)
print("       DATASET STATISTICS CHECK       ")
print("="*50)
print(f"Original Count: {len(ds_orig)}")
print(f"New Count:      {len(ds_new)}")

if len(ds_orig) == len(ds_new):
    print("✅ [PASS] Dataset sizes match (1:1 mapping preserved).")
else:
    print("❌ [FAIL] Dataset sizes do not match!")

# ==========================================
# 4. 상세 내용 비교 (Sample Inspection)
# ==========================================
def compare_sample(idx):
    print("\n" + "="*80)
    print(f"       COMPARING SAMPLE INDEX: {idx}       ")
    print("="*80)
    
    orig_text = ds_orig[idx]['prompt_text']
    new_text = ds_new[idx]['prompt_text']
    
    # 1. Raw String 비교 (개행문자, 특수문자 확인용)
    print("\n[1] Raw String Comparison (To see \\n and hidden chars)")
    print("-" * 40)
    print(f"🔴 ORIGINAL (Raw):\n{repr(orig_text)}")
    print("-" * 40)
    print(f"🟢 CORRECTED (Raw):\n{repr(new_text)}")
    
    # 2. 구조적 차이 분석
    print("\n[2] Structure Analysis")
    print("-" * 40)
    
    # Check Original
    has_inst_orig = "[INST]" in orig_text
    has_double_newline_orig = "\n\n" in orig_text
    print(f"🔴 Original Check -> Has [INST]: {has_inst_orig} | Has \\n\\n: {has_double_newline_orig}")
    
    # Check New
    has_inst_new = "[INST]" in new_text
    has_double_newline_new = "\n\n" in new_text
    starts_correctly = new_text.startswith("<s>[INST] You are a helpful assistant")
    ends_correctly = new_text.strip().endswith("[/INST]")
    
    print(f"🟢 Corrected Check -> Has [INST]: {has_inst_new} | Has \\n\\n: {has_double_newline_new}")
    print(f"                   -> Starts Valid: {starts_correctly} | Ends Valid: {ends_correctly}")

    # 3. 텍스트 렌더링 (실제 LLM이 보는 모습)
    print("\n[3] Rendered View (How it looks)")
    print("-" * 40)
    print("🟢 CORRECTED VERSION:")
    print(new_text)
    print("-" * 40)

# ==========================================
# 5. 실행 (앞부분 샘플과 중간 샘플 비교)
# ==========================================

# 첫 번째 샘플 비교
compare_sample(0)

# (옵션) 중간 임의 샘플 비교
if len(ds_new) > 10:
    compare_sample(10)

print("\n" + "="*50)
print("       COMPARISON FINISHED       ")
print("="*50)

Loading Original: /home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer_bace
Loading Corrected: /home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer_bace_remake

       DATASET STATISTICS CHECK       
Original Count: 152
New Count:      152
✅ [PASS] Dataset sizes match (1:1 mapping preserved).

       COMPARING SAMPLE INDEX: 0       

[1] Raw String Comparison (To see \n and hidden chars)
----------------------------------------
🔴 ORIGINAL (Raw):
'<s>[INST] You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule captioning, molecule generation. \n\nUsing <SELFIES> [C][C][Branch1][C][C][Branch1][C][C][C][C][=C][C][Branch2][Branch1][=N][C][NH2+1][C][C][S][=Branch1][C][=O][=Branch1][C][=O][C][C][Branch2][Ring2][Branch2][C

In [15]:
from datasets import load_from_disk
original_path = '/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer_bace'
corrected_path = '/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer_bace_remake'
download_path = '/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_InstructGraph_bace'
download_bace_path = '/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415'
ds_orig = sorted(load_from_disk(original_path), key = lambda x: x['input_mol_string'])
ds_new = sorted(load_from_disk(corrected_path), key = lambda x: x['input_mol_string'])
ds_down = sorted(load_from_disk(download_path), key = lambda x: x['input_mol_string'])
ds_down_bace = sorted(load_from_disk(download_bace_path), key = lambda x: x['input_mol_string'])[0]

In [17]:
for orig, new, down, down_bace in zip(ds_orig, ds_new, ds_down, ds_down_bace):
    print(sorted(orig.items()))
    print(sorted(new.items()))
    print(sorted(down.items()))
    print(sorted(ds_down_bace.items()))
    break

[('additional_edge_attr', [[0, 0, 0], [0, 0, 0]]), ('additional_edge_index', [[0, 1], [1, 0]]), ('additional_x', [[5, 0, 4, 5, 3, 0, 2, 0, 0], [5, 0, 4, 5, 3, 0, 2, 0, 0]]), ('edge_attr', [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 1], [1, 0, 1], [0, 0, 1], [0, 0, 1], [3, 0, 1], [3, 0, 1], [3, 0, 1], [3, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 1], [1, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [3, 0, 1], [3, 0, 1], [3, 0, 1], [3, 0, 1], [3, 0, 1], [3, 0, 1], [3, 0, 1], [3, 0, 1], [3, 0, 1], [3, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 1], [1, 0, 1], [3, 0, 1], [3, 0, 1], [3, 0, 1], [3, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0